In [38]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [39]:
data = pd.read_csv('data/heart.csv')
index=13
inputs = data.iloc[index, :-1].to_dict()
inputs

{'Age': 49,
 'Sex': 'M',
 'ChestPainType': 'ASY',
 'RestingBP': 140,
 'Cholesterol': 234,
 'FastingBS': 0,
 'RestingECG': 'Normal',
 'MaxHR': 140,
 'ExerciseAngina': 'Y',
 'Oldpeak': 1.0,
 'ST_Slope': 'Flat'}

In [40]:
data.iloc[index, -1:].to_dict()

{'HeartDisease': 1}

In [41]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [42]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://first-pipeline-production.up.railway.app/v1/models/cc-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json().get("predictions")
    if prediction:
        prediction_value = prediction[0][0]
        result = "Heart Failure" if prediction_value > 0.5 else "Healty"
    else:
        result = "Error: No predictions found in response."
    return result


In [43]:
make_predictions(inputs)

'Heart Failure'